<a href="https://colab.research.google.com/github/YoungriKIM/chch/blob/main/official_introductory_tutorial/oit_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 3강. 데이터 불러오기 + 커스터마이징

In [2]:
# 라이브러리 불러오기
import torch
import torchvision
# 데이터를 불러오면서 바로 전처리를 할 수 있는 라이브러리 transforms
import torchvision.transforms as tr 
# DataLoader: 데이터를 배치사이즈 형태로 만들어 학습에 이용할 수 있게함
from torch.utils.data import DataLoader, Dataset
import numpy as np

### 파이토치에서 제공하는 데이터 불러오기
---

In [3]:
# transform 을 미리 정의하자
# 전처리 할 때 compose 안에 있는 순서대로 진행
# Resize : 8,8로 리사이즈 , ToTensor: 텐서로 변환
transf = tr.Compose([tr.Resize(8), tr.ToTensor()])

In [4]:
trainset = torchvision.datasets.CIFAR10(root='D:/chchdata/dataset', train=True, download=True, transform=transf)
testset = torchvision.datasets.CIFAR10(root='D:/chchdata/dataset', train=False, download=True, transform=transf)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# 트레인 셋의 첫번째를 불러와서 사이즈 확인
trainset[0][0].size()
# 3: 컬러라 3채널
# 8,8 리사이즈 한 것

torch.Size([3, 8, 8])

In [6]:
# 배치 사이즈 별로 데이터를 불러오자
trainloader = DataLoader(trainset, batch_size=50, shuffle=True, num_workers=1)
TestLoader = DataLoader(testset, batch_size=50, shuffle=True, num_workers=1)

In [7]:
# dataloader의 길이를 확인해서 배치 사이즈로 잘 잘라졌는지 확인
len(trainloader)

# 전체 데이터의 수가 50,000개이기 때문에 배치 사이즈를 50으로 했을 때 길이가 1,000이 나오게 된다.

1000

In [8]:
# train 안의 실제 값을 확인하고 싶다면
dataiter = iter(trainloader)
images, labels = dataiter.next() # next : 한 묶음을 가져오겠다.

# 이렇게 불러온 이미지 사이즈를 확인하면
print(images.size())
# torch.Size([50, 3, 8, 8]) : 50개의 배치 사이즈로 한 묶음인 3채널의 8,8 크기의 이미지

torch.Size([50, 3, 8, 8])


---
### 개인 데이터 사용하기  
---

In [9]:
# ImageFolder를 이용한 데이터 불러오기 
genderdir = 'D:/aidata/gender' # 안에 female 과 male 이 나뉘어져 정리되어 있는 상태
transf = tr.Compose([tr.Resize(16), tr.ToTensor()]) # 전처리를 미리 정의
trainset = torchvision.datasets.ImageFolder(root = genderdir, transform=transf) # 이미지폴더를 이용해 루트안의 데이터를 transf로 지정한 전처리 해서 불러오기
trainloader = DataLoader(trainset, batch_size=1, shuffle=False, num_workers=1)
print(len(trainloader))
# 한 폴더 당 10개씩이고 batch_size가 1이니 총 20

20


In [10]:
# 사이즈도 확인해보자
trainset[0][0].size()
# resize한 16으로 나옴

torch.Size([3, 16, 16])

---
### 개인 데이터 사용 방법
---

In [11]:
# numpy로 데이터가 들어왔다고 가정
train_images = np.random.randint(256, size=(20, 32, 32, 3))
train_labels = np.random.randint(2, size=(20, 1))

print(train_images.shape, train_labels.shape)

(20, 32, 32, 3) (20, 1)


In [12]:
# 외부의 데이터를 상속 받을 수 있도록 하는 클래스 정의
# 원하는 전처리 형태로 바꿔서 사용 가능

class TensorData(Dataset):
    
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data) # FloatTensor : 2비트의 부동 소수점
        self.x_data = self.x_data.permute(0,3,1,2) # permute를 이용해 토치에 맞게 데이터 순서를 바꿈
        self.y_data = torch.LongTensor(y_data) # LongTensor : 64비트의 부호 있는 정수
        self.len = self.y_data.shape[0]
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [13]:
train_data = TensorData(train_images, train_labels) # 외부 데이터를 텐서 형태로 불러오기
train_loader = DataLoader(train_data, batch_size = 10, shuffle=True)    # 전처리를 거쳐서 배치사이즈 별로 데이터셋 만들기

In [14]:
# 생성된 데이터셋의 사이즈를 확인하자
train_data[0][0].size()
# torch.Size([3, 32, 32]) : 3 채널인 사이즈 32,32인 데이터

torch.Size([3, 32, 32])

In [15]:
# iter를 이용한 사이즈 확인
dataiter = iter(train_loader)
images, labels = dataiter.next()

print(images.size())
# torch.Size([10, 3, 32, 32]) : 10개 배치 사이즈로 묶인 3 채널에 사이즈 32,32인 데이터

torch.Size([10, 3, 32, 32])


### ImageFolder를 자주 쓰지 않는 이유:
디렉토리를 함부로 바꿀 수 없는 데이터들이 있어서, 전처리가 제한 적이어서.

---

### 개인 데이터에 transform 까지 이용하기
---

In [20]:
# TensorData 클래스를 이용하면 된다.

class MyDataset(Dataset):

    def __init__(self, x_data, y_data, transform=None):  # 전처리까지 하기 위해 transform 을 추가

        self.x_data = x_data    # 텐서로 변환하는 것은 TOTensor 클래스로 옮김
        self.y_data = y_data
        self.transform = transform
        self.len = len(y_data)

    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]

        if self.transform:  # 있을 때만 진행, None일 경우 생략
            sample = self.transform(sample)
        
        return sample
    
    def __len__(self):
        return self.len

class ToTensor:     # 콜 함수를 만들어 샘플을 이 클래스를 통해 받는다
    def __call__(self, sample):
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2,0,1)
        return inputs, torch.LongTensor(labels)

# 다른 예제로 하나 더 만들어 보자
class LinearTensor:
    
    def __init__(self, slope = 1, bias = 0):
        self.slope = slope
        self.bias = bias

    def __call__(self, sample): # call 함수 부분에 원하는 계산들을 작성한다.
        inputs, labels = sample
        inputs = self.slope * inputs + self.bias

        return inputs, labels

In [65]:
# 사용하는 방법은 위와 동일하다
trans = tr.Compose([ToTensor(), LinearTensor(2,5)]) # 여기서 사용한 ToTensor는 바로 위에서 정의한 ToTensor이다.
dsl = MyDataset(train_images, train_labels, transform = trans)
train_loader1 = DataLoader(dsl, batch_size=10, shuffle = True)

In [66]:
# 데이터의 형태를 확인해보자
first_data = dsl[0]
featrues, labels = first_data
print(type(featrues), type(labels))

# <class 'torch.Tensor'> <class 'torch.Tensor'>
# 넘파이였던 형태가 토치로 변환되었음을 확인

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [69]:
# 데이터 값 확인
dataiter1 = iter(train_loader1)
images1, labels1 = dataiter1.next()

print(images1.size())
# torch.Size([10, 3, 32, 32])

torch.Size([10, 3, 32, 32])


In [73]:
# 안의 값도 다 확인해보자
images1[0][:5]

tensor([[[341., 161., 505.,  ..., 403.,  39., 271.],
         [ 35., 301., 407.,  ..., 269., 413.,  43.],
         [235., 435., 379.,  ..., 293.,  59., 195.],
         ...,
         [347., 155., 443.,  ..., 277., 361., 115.],
         [143.,  89.,  73.,  ..., 131., 131., 193.],
         [351.,  27., 215.,  ..., 289.,  15.,  85.]],

        [[301.,  33., 241.,  ...,  67., 223., 419.],
         [ 19., 441., 103.,  ..., 123., 417., 255.],
         [113., 393., 409.,  ..., 323.,  25., 225.],
         ...,
         [ 77., 317., 327.,  ..., 501., 347., 123.],
         [439., 235., 269.,  ..., 515., 461., 313.],
         [383., 363., 129.,  ...,  91., 331.,  29.]],

        [[395., 121., 149.,  ..., 439., 299., 341.],
         [ 71.,   9., 469.,  ..., 365.,  55., 403.],
         [133., 199., 429.,  ..., 513., 239., 397.],
         ...,
         [139., 213.,  41.,  ...,  41.,  89.,  25.],
         [357., 263., 445.,  ..., 393., 481., 339.],
         [511., 447., 155.,  ..., 497., 405., 471.]]]

---
### torch에서 제공하는 ToTensor 등의 전처리 기능을 사용하지 않고 왜 새로 만들어서 사용하는가?
타입에러 등 제공하는 옵션과 다른 경우가 많아서..  
그런데도 불구하고 제공하는 기능을 활용하고 싶다면 아래와 같이 하자


In [58]:
class MyDataset(Dataset):

    def __init__(self, x_data, y_data, transform = None):
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.len = len(y_data)
    
    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]

        if self.transform:
            sample = self.transform(sample)
        
        return sample
    
    def __len__(self):
        return self.len
    
class MyTransform:

    def __call__(self, sample):
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2,0,1)
        labels = torch.FloatTensor(labels)

        transf = tr.Compose([tr.ToPILImage(), tr.Resize(128), tr.ToTensor(), tr.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        final_output = transf(inputs)

        return final_output, labels

In [59]:
# 정의한 함수를 이용해 데이터를 처리하자
ds2 = MyDataset(train_images, train_labels, transform=MyTransform())
train_loader2 = DataLoader(ds2, batch_size=10, shuffle=True)

In [60]:
# 데이터를 형태를 확인해보자
first_data = ds2[0]
features, labels = first_data
print(type(features), type(labels))

# <class 'torch.Tensor'> <class 'torch.Tensor'>
# 텐서형태로 바뀐 것 확인

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [64]:
# 데이터 사이즈 확인
dataiter2 = iter(train_loader2)
images2, labels2 = dataiter2.next()

print(images2.size())

# torch.Size([10, 3, 128, 128])
# 배치사이즈 10에 3채널인 128,128인 이미지

torch.Size([10, 3, 128, 128])


In [74]:
#  normalize 되었는지 값을 확인해보자
images2[0][:5]

# 확인

tensor([[[ 0.7725,  0.7725,  0.7647,  ...,  0.4980,  0.6706,  0.6706],
         [ 0.7725,  0.7725,  0.7647,  ...,  0.4980,  0.6706,  0.6706],
         [ 0.5843,  0.5843,  0.5843,  ...,  0.5216,  0.7098,  0.7098],
         ...,
         [-0.0824, -0.0824, -0.0824,  ..., -0.4275, -0.4118, -0.4118],
         [-0.1843, -0.1843, -0.1608,  ..., -0.5765, -0.5765, -0.5765],
         [-0.1843, -0.1843, -0.1608,  ..., -0.5765, -0.5765, -0.5765]],

        [[-0.7725, -0.7725, -0.6000,  ..., -0.8039, -0.8039, -0.8039],
         [-0.7725, -0.7725, -0.6000,  ..., -0.8039, -0.8039, -0.8039],
         [-0.6078, -0.6078, -0.4667,  ..., -0.6157, -0.6078, -0.6078],
         ...,
         [ 0.5373,  0.5373,  0.5765,  ...,  0.5216,  0.6941,  0.6941],
         [ 0.5765,  0.5765,  0.6235,  ...,  0.4824,  0.6549,  0.6549],
         [ 0.5765,  0.5765,  0.6235,  ...,  0.4824,  0.6549,  0.6549]],

        [[-0.1059, -0.1059, -0.0510,  ..., -0.5059, -0.7176, -0.7176],
         [-0.1059, -0.1059, -0.0510,  ..., -0